In [28]:
import cv2
from cv2 import cuda
from math import*
import numpy as np
from numba import jit,njit
import time
import pandas as pd
import matlab.engine
# Khởi động matlab engine
eng = matlab.engine.start_matlab()
print('MALAB ENGINE FINISHED BEGINNING !!!')

MALAB ENGINE FINISHED BEGINNING !!!


In [29]:
def calculate_trajectory(map,startPoint,goalPoint):
    startPoint = matlab.double(startPoint) # [meters, meters, radians]
    goalPoint  = matlab.double(goalPoint)
    map  = matlab.double(map.tolist())
    result = eng.Hybrid_Astar(map,startPoint,goalPoint)
    result = np.array(result)
    point = result[:,:3]
    point[:,2] = (point[:,2]*180/pi)-90
    direction = result[:,3]
    return point, direction

In [39]:
local_map =np.array(pd.read_csv("map_test.csv"))

local_map[0,90:98]=1
local_map[0,102:110]=1
goalPose = [100, 198, 1.5707963267948966]
s_state = [100, 100, (0+90)*pi/180]

@jit(nopython = True)
def check_collision_goal (loc_map, goal_x, goal_y):
    loc_x_check = int(goal_x)
    loc_y_check = int(200-goal_y)
    loc_y_up = max(min(loc_y_check-10, 200), 0)
    loc_y_down = max(min(loc_y_check+11, 200), 0)

    
    chk_right = 0
    chk_left = 0
    max_cnt_right = 200 - loc_x_check
    max_cnt_left  = loc_x_check 
    collision = False
    # dò sang bên phải 
    for cnt_right in range(0,max_cnt_right):
        column = loc_map[loc_y_up:loc_y_down,loc_x_check+cnt_right]
        if np.all(column == 0):
            chk_right = chk_right + 1 
            if chk_right == 10:
                break
        else: 
            chk_left = 0
    for cnt_left in range(0,max_cnt_left):
        column = loc_map[loc_y_up:loc_y_down,loc_x_check-cnt_left]
        if np.all( column == 0):
            chk_left = chk_left + 1 
            if chk_left == 10:
                break
        else: 
            chk_right = 0
    if cnt_right > max_cnt_right-5  and  cnt_left>max_cnt_left-5 :
        collision = True
        x_out = 0
        y_out = 0
    
    elif cnt_right > max_cnt_right-5 :
        x_out = goal_x - cnt_left + chk_left
        y_out = goal_y


    elif cnt_left > max_cnt_left-5:
        x_out = goal_x + cnt_right - chk_right
        y_out = goal_y

    else:
        if cnt_right <= cnt_left:
            x_out = goal_x + cnt_right - chk_right
            y_out = goal_y

        else:
            x_out = goal_x - cnt_left + chk_left
            y_out = goal_y


    map = np.copy(loc_map)
    return map,collision, x_out,y_out
start = time.time()


chk_local_map, check_colision, checked_goal_x, checked_goal_y = check_collision_goal (loc_map = local_map, goal_x = goalPose[0], goal_y = goalPose[1])
goalChecked = [int(checked_goal_x),int(checked_goal_y), goalPose[2]]
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))
print(goalChecked)
print(check_colision)
# list_state, list_direction = calculate_trajectory(map = chk_local_map ,startPoint = s_state, goalPoint = goalChecked)


3
17 17
Elapsed (with compilation) = 0.5999853610992432
[117, 198, 1.5707963267948966]
False


In [31]:
column = local_map[0:10][105]
print()

IndexError: index 105 is out of bounds for axis 0 with size 10

In [ ]:
list_state = np.array([[ 1.00000000e+02,  100.000000,  0],
                    [ 101.177506,  114.950935, -4.92665016],
                    [ 102.643622,  131.959477, -4.92665016],
                    [ 104.109738,  148.968020, -4.92665016],
                    [ 105.575854,  165.976562, -4.92665016],
                    [ 107.041970,  182.985105, -4.92665016],
                    [ 108.397387,  200.000000,  1.42108547]])




In [ ]:
start = time.time()

end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

4


array([[140, 710],
       [141, 696],
       [142, 679],
       [144, 662],
       [145, 645],
       [147, 628],
       [148, 610]])